In [0]:
# ============================================================================
# Camada Gold - Análise Temporal - Magic: The Gathering
# Pipeline 100% PySpark DataFrame API
# ============================================================================

# ============================================================================
# BIBLIOTECAS UTILIZADAS
# ============================================================================
import logging
from datetime import datetime
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
from pyspark.sql.utils import AnalysisException

# ============================================================================
# IMPORTAÇÃO DO MÓDULO UTILITÁRIO
# ============================================================================
%run ./gold_utils

# ============================================================================
# CONFIGURAÇÃO COM MÓDULO UTILITÁRIO
# ============================================================================
TABLE_NAME = "TB_ANALISE_TEMPORAL"

def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# Inicializar processador com configuração manual (recomendado para desenvolvimento)
config = create_manual_config(
    catalog_name="magic_the_gathering",  # Catalog correto do projeto
    s3_bucket=get_secret("s3_bucket"),  # Ajuste conforme seu bucket
    s3_gold_prefix="magic_the_gathering/gold"
)
processor = GoldTableProcessor(TABLE_NAME, config)
spark = processor.spark

# Parâmetros de análise temporal
JANELA_SAZONALIDADE = 365  # dias para análise sazonal
JANELA_CICLO_VIDA = 730   # dias para análise de ciclo de vida (2 anos)

# ============================================================================
# Funções Auxiliares
# ============================================================================
def extrair_componentes_temporais(df):
    """
    Extrai componentes temporais baseados na data de release do set
    Usa RELEASE_YEAR e RELEASE_MONTH (dados de negócio reais)
    """
    df_temporal = df.withColumn("DATA_REF", make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1))) \
                    .withColumn("ANO", col("cards.RELEASE_YEAR")) \
                    .withColumn("MES", col("cards.RELEASE_MONTH")) \
                    .withColumn("DIA_SEMANA", dayofweek(make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1)))) \
                    .withColumn("DIA_ANO", dayofyear(make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1)))) \
                    .withColumn("TRIMESTRE", quarter(make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1)))) \
                    .withColumn("SEMANA_ANO", weekofyear(make_date(col("cards.RELEASE_YEAR"), col("cards.RELEASE_MONTH"), lit(1))))
    # Classificações temporais
    df_temporal = df_temporal.withColumn("NOME_MES",
        when(col("MES") == 1, "Janeiro")
        .when(col("MES") == 2, "Fevereiro")
        .when(col("MES") == 3, "Março")
        .when(col("MES") == 4, "Abril")
        .when(col("MES") == 5, "Maio")
        .when(col("MES") == 6, "Junho")
        .when(col("MES") == 7, "Julho")
        .when(col("MES") == 8, "Agosto")
        .when(col("MES") == 9, "Setembro")
        .when(col("MES") == 10, "Outubro")
        .when(col("MES") == 11, "Novembro")
        .when(col("MES") == 12, "Dezembro"))
    df_temporal = df_temporal.withColumn("NOME_DIA_SEMANA",
        when(col("DIA_SEMANA") == 1, "Domingo")
        .when(col("DIA_SEMANA") == 2, "Segunda")
        .when(col("DIA_SEMANA") == 3, "Terça")
        .when(col("DIA_SEMANA") == 4, "Quarta")
        .when(col("DIA_SEMANA") == 5, "Quinta")
        .when(col("DIA_SEMANA") == 6, "Sexta")
        .when(col("DIA_SEMANA") == 7, "Sábado"))
    return df_temporal

def calcular_idade_carta(df):
    """
    Calcula a idade da carta desde o lançamento
    Baseado na data de release do set vs data atual
    """
    df_idade = df.withColumn("IDADE_CARTA_DIAS",
                            datediff(current_date(), col("DATA_REF")))
    df_idade = df_idade.withColumn("FASE_CICLO_VIDA",
        when(col("IDADE_CARTA_DIAS") <= 30, "Lançamento")
        .when(col("IDADE_CARTA_DIAS") <= 90, "Crescimento")
        .when(col("IDADE_CARTA_DIAS") <= 365, "Maturidade")
        .when(col("IDADE_CARTA_DIAS") <= 730, "Estabilidade")
        .otherwise("Declínio"))
    return df_idade



In [0]:
# ============================================================================
# CARREGAMENTO DE DADOS SILVER COM MÓDULO
# ============================================================================
dfs = processor.load_silver_data(['cards', 'prices', 'sets'])
df_cards = dfs['cards']
df_prices = dfs['prices']
df_sets = dfs['sets']

# Join Principal com Dados Temporais
# Cards tem: NME_CARD, COD_SET, NME_SET, DT_INGESTION, RELEASE_YEAR, RELEASE_MONTH
# Prices tem: NME_CARD, COD_SET, VLR_USD, VLR_EUR, VLR_TIX, DT_INGESTION
# Sets tem: COD_SET, NME_SET (usar chave COD_SET)
df_base = df_cards.join(df_prices, ["NME_CARD", "COD_SET"], "inner") \
                  .join(df_sets, ["COD_SET"], "left")
# Para evitar ambiguidade, renomeie a coluna do set:
df_base = df_base.withColumn("NME_SET_SET", col("sets.NME_SET"))

# ============================================================================
# Extração de Componentes Temporais
# ============================================================================
df_temporal = extrair_componentes_temporais(df_base)
df_temporal = calcular_idade_carta(df_temporal)

In [0]:
# ============================================================================
# 1. ANÁLISE DE SAZONALIDADE POR MÊS
# ============================================================================
df_sazonalidade_mensal = df_temporal.groupBy("MES", "NOME_MES", "NME_SET_SET") \
    .agg(
        avg("VLR_USD").cast("float").alias("PRECO_MEDIO_MES"),
        stddev("VLR_USD").cast("float").alias("VOLATILIDADE_MES"),
        count("*").cast("int").alias("QTD_OBSERVACOES"),
        countDistinct("NME_CARD").cast("int").alias("QTD_CARTAS_ATIVAS"),
        sum("VLR_USD").cast("float").alias("VOLUME_TOTAL_MES")
    )
window_anual = Window.partitionBy("NME_SET_SET")
df_sazonalidade_mensal = df_sazonalidade_mensal.withColumn("PRECO_MEDIO_ANUAL",
    avg("PRECO_MEDIO_MES").over(window_anual))
df_sazonalidade_mensal = df_sazonalidade_mensal.withColumn("INDICE_SAZONAL",
    (col("PRECO_MEDIO_MES") / col("PRECO_MEDIO_ANUAL")).cast("float"))

# ============================================================================
# 2. ANÁLISE DE SAZONALIDADE POR DIA DA SEMANA
# ============================================================================
df_sazonalidade_semanal = df_temporal.groupBy("DIA_SEMANA", "NOME_DIA_SEMANA", "NME_SET_SET") \
    .agg(
        avg("VLR_USD").cast("float").alias("PRECO_MEDIO_DIA"),
        stddev("VLR_USD").cast("float").alias("VOLATILIDADE_DIA"),
        count("*").cast("int").alias("QTD_OBSERVACOES_DIA")
    )

# ============================================================================
# 3. CICLO DE VIDA DAS CARTAS
# ============================================================================
df_ciclo_vida = df_temporal.groupBy("FASE_CICLO_VIDA", "NME_CARD_TYPE", "cards.NME_RARITY") \
    .agg(
        avg("VLR_USD").cast("float").alias("PRECO_MEDIO_FASE"),
        count("*").cast("int").alias("QTD_CARTAS_FASE"),
        avg("IDADE_CARTA_DIAS").cast("float").alias("IDADE_MEDIA_FASE"),
        expr("percentile_approx(VLR_USD, 0.5)").cast("float").alias("PRECO_MEDIANO_FASE"),
        max("VLR_USD").cast("float").alias("PRECO_MAXIMO_FASE")
    ) \
    .withColumnRenamed("cards.NME_RARITY", "NME_RARITY")

# ============================================================================
# 4. IMPACTO DE LANÇAMENTOS (Release Events)
# ============================================================================
window_lancamento = Window.partitionBy("COD_SET").orderBy("DATA_REF")
df_impacto_lancamento = df_temporal.withColumn("DIAS_DESDE_LANCAMENTO",
    datediff(col("cards.DT_INGESTION"), col("DATA_REF")))
df_impacto_lancamento = df_impacto_lancamento.filter(
    (col("DIAS_DESDE_LANCAMENTO") >= -30) & 
    (col("DIAS_DESDE_LANCAMENTO") <= 30)
)
df_impacto_lancamento = df_impacto_lancamento.withColumn("PERIODO_LANCAMENTO",
    when(col("DIAS_DESDE_LANCAMENTO") < 0, "Pre-Release")
    .when(col("DIAS_DESDE_LANCAMENTO") == 0, "Release")
    .otherwise("Post-Release"))
df_impacto_lancamento = df_impacto_lancamento.withColumn("NME_SET_SET", col("sets.NME_SET"))
df_impacto_agregado = df_impacto_lancamento.groupBy("COD_SET", "NME_SET_SET", "PERIODO_LANCAMENTO") \
    .agg(
        avg("VLR_USD").cast("float").alias("PRECO_MEDIO_PERIODO"),
        count("*").cast("int").alias("QTD_CARTAS_PERIODO"),
        sum("VLR_USD").cast("float").alias("VOLUME_PERIODO")
    )

# ============================================================================
# 5. ANÁLISE DE CORRELAÇÃO TEMPORAL (Price Momentum)
# ============================================================================
window_momentum = Window.partitionBy("NME_CARD", "COD_SET").orderBy("DATA_REF")
df_momentum = df_temporal.withColumn("PRECO_7D_ATRAS", lag("VLR_USD", 7).over(window_momentum)) \
                        .withColumn("PRECO_30D_ATRAS", lag("VLR_USD", 30).over(window_momentum))
df_momentum = df_momentum.withColumn("MOMENTUM_7D",
    when(col("PRECO_7D_ATRAS").isNotNull() & (col("PRECO_7D_ATRAS") != 0),
         ((col("VLR_USD") - col("PRECO_7D_ATRAS")) / col("PRECO_7D_ATRAS")).cast("float"))
    .otherwise(lit(None)))
df_momentum = df_momentum.withColumn("MOMENTUM_30D",
    when(col("PRECO_30D_ATRAS").isNotNull() & (col("PRECO_30D_ATRAS") != 0),
         ((col("VLR_USD") - col("PRECO_30D_ATRAS")) / col("PRECO_30D_ATRAS")).cast("float"))
    .otherwise(lit(None)))
df_momentum = df_momentum.withColumn("CLASSIFICACAO_MOMENTUM_7D",
    when(col("MOMENTUM_7D") > 0.1, "Forte Alta")
    .when(col("MOMENTUM_7D") > 0.05, "Alta Moderada")
    .when(col("MOMENTUM_7D") > -0.05, "Estável")
    .when(col("MOMENTUM_7D") > -0.1, "Baixa Moderada")
    .otherwise("Forte Baixa"))

# ============================================================================
# 6. DETECÇÃO DE PADRÕES CÍCLICOS
# ============================================================================
df_ciclos = df_temporal.groupBy("NME_CARD", "COD_SET", "DIA_ANO") \
    .agg(avg("VLR_USD").alias("PRECO_MEDIO_DIA_ANO"))
window_picos = Window.partitionBy("NME_CARD", "COD_SET").orderBy("DIA_ANO")
df_ciclos = df_ciclos.withColumn("PRECO_ANTERIOR", lag("PRECO_MEDIO_DIA_ANO").over(window_picos)) \
                    .withColumn("PRECO_POSTERIOR", lead("PRECO_MEDIO_DIA_ANO").over(window_picos))
df_ciclos = df_ciclos.withColumn("IS_PICO_SAZONAL",
    when((col("PRECO_MEDIO_DIA_ANO") > col("PRECO_ANTERIOR")) & 
         (col("PRECO_MEDIO_DIA_ANO") > col("PRECO_POSTERIOR")), lit(True))
    .otherwise(lit(False)))

# ============================================================================
# 7. AGREGAÇÃO FINAL TEMPORAL CONSOLIDADA
# ============================================================================
df_temporal_clean = df_momentum.select(
    "NME_CARD", "COD_SET", col("sets.NME_SET").alias("NME_SET_SET"), "NME_CARD_TYPE",
    "ANO", "MES", "NOME_MES", "TRIMESTRE", "VLR_USD", "IDADE_CARTA_DIAS", 
    "FASE_CICLO_VIDA", "MOMENTUM_7D", "MOMENTUM_30D", "DATA_REF"
)
df_analise_temporal_final = df_temporal_clean.groupBy(
    "NME_CARD", "COD_SET", "NME_SET_SET", "NME_CARD_TYPE",
    "ANO", "MES", "NOME_MES", "TRIMESTRE"
).agg(
    avg("VLR_USD").cast("float").alias("PRECO_MEDIO_PERIODO"),
    min("VLR_USD").cast("float").alias("PRECO_MINIMO_PERIODO"),
    max("VLR_USD").cast("float").alias("PRECO_MAXIMO_PERIODO"),
    stddev("VLR_USD").cast("float").alias("VOLATILIDADE_PERIODO"),
    count("*").cast("int").alias("QTD_OBSERVACOES"),
    avg("IDADE_CARTA_DIAS").cast("float").alias("IDADE_MEDIA_CARTA"),
    first("FASE_CICLO_VIDA").alias("FASE_CICLO_VIDA"),
    avg("MOMENTUM_7D").cast("float").alias("MOMENTUM_MEDIO_7D"),
    avg("MOMENTUM_30D").cast("float").alias("MOMENTUM_MEDIO_30D"),
    first("DATA_REF").alias("DATA_REF")
)



In [0]:
# ============================================================================
# ESCRITA DA TABELA PRINCIPAL COM MÓDULO
# ============================================================================
processor.save_gold_table(df_analise_temporal_final, partition_cols=["DATA_REF", "ANO"])

# ============================================================================
# FINALIZAÇÃO
# ============================================================================
print("TB_ANALISE_TEMPORAL modularizada criada com sucesso!")